In [7]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools
from sklearn.ensemble import ExtraTreesClassifier


In [8]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [9]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5117 entries, 0 to 5116
Columns: 920 entries, Unnamed: 0 to nk_feat_378
dtypes: float64(919), int64(1)
memory usage: 35.9 MB


In [10]:
#save feature name
feature_name = x_train.columns

In [11]:
selector = SelectKBest(score_func=f_classif, k=150)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)

In [12]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [13]:
#random forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=0)

In [14]:
#cross validation score
scores = cross_val_score(clf, x_train, np.ravel(y_train), cv=10, scoring='f1_micro',n_jobs=-1)
print(scores.mean())
print(scores.std())
#0.6880920529598826 / 0.01306019561569018 MinMaxScaler
#0.6912193462573386 / 0.011284110708011923 new preprocessing
#0.6984512597847358 / 0.012100579167071735 new new preprocessing
#0.6990349039872799 / 0.009122000817677589 378 nk feat

0.6990349039872799
0.009122000817677589


In [15]:
clf.fit(x_train, np.ravel(y_train))

RandomForestClassifier(random_state=0)

In [16]:
#feature importance
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
# Print the feature ranking
print("Feature ranking:")
for f in range(x_train.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, feature_name[indices[f]], importances[indices[f]]))
    

Feature ranking:
1. feature nk_feat_274 (0.006229)
2. feature nk_feat_277 (0.005220)
3. feature nk_feat_338 (0.005064)
4. feature nk_feat_51 (0.005057)
5. feature nk_feat_54 (0.004876)
6. feature nk_feat_49 (0.004760)
7. feature nk_feat_52 (0.004666)
8. feature nk_feat_337 (0.004573)
9. feature nk_feat_80 (0.004225)
10. feature nk_feat_275 (0.004141)
11. feature nk_feat_77 (0.004070)
12. feature nk_feat_175 (0.003774)
13. feature nk_feat_309 (0.003415)
14. feature nk_feat_95 (0.003410)
15. feature nk_feat_172 (0.003292)
16. feature nk_feat_93 (0.003290)
17. feature nk_feat_99 (0.002937)
18. feature nk_feat_234 (0.002835)
19. feature nk_feat_50 (0.002819)
20. feature nk_feat_92 (0.002738)
21. feature mu_49 (0.002694)
22. feature nk_feat_55 (0.002692)
23. feature nk_feat_174 (0.002645)
24. feature nk_feat_85 (0.002617)
25. feature nk_feat_239 (0.002598)
26. feature nk_feat_100 (0.002499)
27. feature nk_feat_156 (0.002499)
28. feature nk_feat_97 (0.002482)
29. feature mu_48 (0.002478)
30.

In [17]:
import pickle
filename = 'models/random_forest_model.sav'
pickle.dump(clf, open(filename, 'wb'))


In [18]:
y_test = clf.predict(x_test)
data_y_test = {
    'id': [i for i in range(len(y_test))],
    'y': y_test
}
df_y_test = pd.DataFrame(data_y_test)
df_y_test.to_csv('outputs/output_random_forest.csv', index=False)

/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 631 features, but RandomForestClassifier is expecting 920 features as input.